In [7]:
import random
# Install required libraries
# Import required libraries
import gymnasium as gym
from gymnasium import spaces
import matplotlib.pyplot as plt
import numpy as np
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from collections import namedtuple, deque
device = torch.device(
    "cuda" if torch.cuda.is_available() else
    "cpu"
)

print(device)
CUDA_LAUNCH_BLOCKING=1

cuda


In [8]:
class Net(nn.Module):

    def __init__(self, obs, action):
        super(Net, self).__init__()
        self.layer1 = nn.Linear(obs, 32)
        self.layer2 = nn.Linear(32, 32)
        self.layer3 = nn.Linear(32, action)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)

In [9]:
class DQN:
  # initialize values
  def __init__(self, N, env):
    # initialize environment
    self.env = env
    state, info = self.env.reset()
    # initialize replay memory to capacity N
    self.pointer = 0
    self.policy_net = Net(len(state), self.env.action_space.n).to(device)
    self.target_net = Net(len(state), self.env.action_space.n).to(device)
    
    self.optimizer = optim.AdamW(self.policy_net.parameters(),amsgrad=True ) #auto learning rate
    # big replay memory:
    self.size = N
    self.state_mem = torch.zeros((N, self.env.observation_space.shape[0]), dtype=torch.float32, device=device)
    self.next_state_mem = torch.zeros((self.size, self.env.observation_space.shape[0]), dtype=torch.float32, device=device)
    self.action_mem = torch.zeros(self.size, dtype=torch.int64, device=device)
    self.reward_mem = torch.zeros(self.size, dtype=torch.float32, device=device)
    self.done_mem = torch.zeros(self.size, dtype=torch.float32, device=device)
    self.pointer = 0

  def append(self, state, action, reward, next_state, done):
    i = self.pointer % self.size #get index
    self.state_mem[i] = state
    self.next_state_mem[i] = next_state
    self.reward_mem[i] = reward 
    self.done_mem[i] = 1 - int(done)
    self.action_mem[i] = action
    self.pointer +=1
    
  def sample(self,batch):
    mem = min(self.pointer,self.size) # get range to choose mem from
    batch = np.random.choice(mem,batch) # choose random indices
    states = self.state_mem[batch]
    next_states = self.next_state_mem[batch]
    actions = self.action_mem[batch]
    rewards = self.reward_mem[batch]
    done = self.done_mem[batch]
    return states, actions, rewards, next_states, done




  
  # Main training function
  def train(self, episodes, epsilon, discount, action_function, greedy):
    total_reward = [0] * episodes  
    # TAU = .0004
    for i in range(episodes):
      if i % 5 == 0:
         self.target_net.load_state_dict(self.policy_net.state_dict())
      # initialize sequence S and preprocessed sequence o
      seq  = [None , None]
      state, info = self.env.reset()
      seq[0] = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
      done = False
      rewards = 0
      eps = epsilon ** i if not greedy else 0
      while not done:
        # Select action
        action_type = action_function(seq[0], eps)
        observation, reward, terminated, truncated, _ = self.env.step(action_type.item())
        state = torch.tensor(observation, device=device, dtype=torch.float32).unsqueeze(0)
        # Set sequence
        seq[1] = state
        self.append(state=seq[0], action=action_type, reward=reward, next_state=seq[1], done=terminated)
        if terminated:
           seq[1] = None
        rewards += reward
        # store transition in replay buffer
        seq[0] = state
        
        self.r(discount)
        done = truncated or terminated
      # Decay epsilon after every episode
      # epsilon *= epsilon
      total_reward[i] = rewards  
    return total_reward
  # Determine the action for the warehouse environment
      
  def action(self, state, epsilon):
    if np.random.rand() < epsilon:
        action_type = self.env.action_space.sample()
    else:
        # select max(Q)
        with torch.no_grad():
            return self.policy_net(state).max(1).indices.view(1, 1) 
    return torch.tensor([[action_type]], device=device, dtype=torch.long)
        
  def r(self,discount):
      BATCH_SIZE = 256
      if self.pointer < BATCH_SIZE:
          return
      # Sample a batch from replay memory
      states, actions, rewards, next_states, dones = self.sample(BATCH_SIZE)
      
      # Convert to tensors
      state_batch = torch.tensor(states, dtype=torch.float32, device=device)
      action_batch = torch.tensor(actions, dtype=torch.int64, device=device).unsqueeze(1)
      reward_batch = torch.tensor(rewards, dtype=torch.float32, device=device)
      next_state_batch = torch.tensor(next_states, dtype=torch.float32, device=device)
      done_mask = torch.tensor(dones, dtype=torch.float32, device=device) # zeros out terminated
      
      # Calculate state-action values
      state_action_values = self.policy_net(state_batch).gather(1, action_batch)
      
      # Calculate next state values
      next_state_values = torch.zeros(BATCH_SIZE, device=device)
      with torch.no_grad():
          next_state_values[done_mask.bool()] = self.target_net(next_state_batch[done_mask.bool()]).max(1).values
      
      # Expected Q values
      expected_state_action_values = (next_state_values * discount) + reward_batch
      
      # Compute loss
      criterion = nn.SmoothL1Loss()
      loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))
      
      # Optimize the model
      self.optimizer.zero_grad()
      loss.backward()
      self.optimizer.step()

  
  # Save the current weights
  def save(self, filename):
    with open(filename, 'wb') as file:
      pickle.dump(self.policy_net.state_dict(), file,protocol=pickle.HIGHEST_PROTOCOL)


In [10]:
#Prints the reward per epsisode graph
def reward_print(reward_per_episode, episodes, info): 
    mins = int(min(reward_per_episode)) - int(min(reward_per_episode)) * (.15)
    maxs = int(max(reward_per_episode)) + int(max(reward_per_episode)) * (.3) 
    plt.figure()
    plt.plot(reward_per_episode)
    plt.xlabel('Episode', fontsize=20)
    plt.ylabel('Cumulative Reward', fontsize=20)
    plt.title(f'Cumulative Reward Per Episode ({info})', fontsize=24)
    plt.xticks([0, episodes * .2, episodes * .4, episodes * .6, episodes * .8, episodes], fontsize=18)
    plt.yticks(fontsize=18)
    plt.ylim(ymin= mins, ymax=maxs)
    plt.xlim(xmin=0, xmax=episodes)
    plt.grid()
    plt.show()

#prints the epsilon decay graph
def ep_decay(eps, episodes):
    epsilon_values = [(eps ** i) * 1 for i in range(episodes)]
    plt.figure()
    plt.plot(epsilon_values, linewidth=4)
    plt.xlabel('Episode', fontsize=20)
    plt.ylabel('Epsilon Value', fontsize=20)
    plt.title(f"Epsilon Decay for {eps}", fontsize=24)
    plt.xticks([0, episodes * .2, episodes * .4, episodes * .6, episodes * .8, episodes], fontsize=18)
    plt.yticks(fontsize=18)
    plt.ylim(ymin=0, ymax=1)
    plt.xlim(xmin=0, xmax=episodes)
    plt.grid()
    plt.show()


In [ ]:
N = 5000
env = gym.make("CartPole-v1")
env.reset()
cart = DQN(N, env)

episodes = 600
epsilon = .8
discount = .99
action = cart.action
total_rewards = cart.train(episodes, epsilon, discount, action, False)
# cart.save("drpreisl_angustsa_assignment2_part2_dqn_cartpole.pickle")
reward_print(total_rewards, episodes, "CartPole")
ep_decay(epsilon, episodes)
total_rewards = cart.train(6, epsilon, discount, action, True)
reward_print(total_rewards, 5, "CartPole")
          # policy_indices = self.policy_net(next_state_batch[done_mask.bool()]).max(1).indices
          # next_state_values[done_mask.bool()] = self.target_net(next_state_batch[done_mask.bool()]).gather(1, policy_indices.unsqueeze(1)).squeeze(1)

C:\Users\corvi\AppData\Local\Temp\ipykernel_42504\1473573199.py:98: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state_batch = torch.tensor(states, dtype=torch.float32, device=device)
C:\Users\corvi\AppData\Local\Temp\ipykernel_42504\1473573199.py:99: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  action_batch = torch.tensor(actions, dtype=torch.int64, device=device).unsqueeze(1)
C:\Users\corvi\AppData\Local\Temp\ipykernel_42504\1473573199.py:100: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  reward_batch = torch.tensor(rewards, dtype=tor

In [ ]:
print(cart.policy_net.state_dict())
# cart.save("drpreisl_angustsa_assignment2_part2_dqn_cartpole.pickle")

OrderedDict({'layer1.weight': tensor([[-0.1163,  0.0632,  0.3759, -0.1103],
        [-0.0551,  0.0651,  0.0883, -0.3082],
        [ 0.1176,  0.1558, -0.3363, -0.2061],
        [ 0.3504,  0.3565,  0.0503,  0.2385],
        [-0.0135, -0.0622, -0.4049,  0.2239],
        [ 0.2374, -0.0444, -0.2621, -0.1243],
        [ 0.0484, -0.2172,  0.1059,  0.0079],
        [-0.3850, -0.2161, -1.0756, -0.1640],
        [ 0.0104, -0.1283, -0.1561, -0.1733],
        [-0.5888, -0.1871,  0.9653, -0.0857],
        [ 0.2225, -0.2026, -0.3287, -0.1285],
        [ 0.0977,  0.1676,  0.9415,  0.2195],
        [-0.0490,  0.0683, -0.0306, -0.0864],
        [-0.5207, -0.1740,  0.0105, -0.0547],
        [ 0.3841, -0.0532, -0.9298, -0.2318],
        [ 0.1776,  0.2235, -0.9380,  0.0086],
        [ 0.1416, -0.2902,  0.2632, -0.0367],
        [-0.1603,  0.1229, -0.1470,  0.0754],
        [ 0.0154,  0.1364,  0.3883,  0.2083],
        [-0.1947,  0.0939, -1.1772, -0.1307],
        [-0.0941, -0.3262, -0.8575, -0.2281],
    